<a href="https://colab.research.google.com/github/MurselErkin/YOLACT/blob/main/YOLACT_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLACT Google Colab 
Gruppe 5





# Initial Setup
First, we install some Python libraries. Google Colab probably has these requirements already, but better to include them than to break later.

In [4]:
# Cython needs to be installed before pycocotools
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
# Downgrade torch to accommodate DCNv2
!pip install torchvision==0.14
!pip install torch==1.13.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.13.0-cp38-cp38-manylinux1_x86_64.whl (890.2 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Clone YOLACT from GitHub
Let's get that github repo! If anything in this notebook breaks, check the repo in case they've made updates.

In [47]:
# Make sure we're in the top folder
%cd /content

# Clone the repo
!git clone https://github.com/dbolya/yolact.git

/content
fatal: destination path 'yolact' already exists and is not an empty directory.


## DCNv2
The DCNv2 external library is needed for this to work, so we'll build it now.

ook.

In [60]:
!apt-get install ninja-build
%cd /content/yolact/external/DCNv2/
!python setup.py clean
!python setup.py build_ext --inplace --debug


Reading package lists... Done
Building dependency tree       
Reading state information... Done
ninja-build is already the newest version (1.8.2-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
/content/yolact/external/DCNv2
running clean
removing 'build/temp.linux-x86_64-3.8' (and everything under it)
removing 'build'
running build_ext
/usr/local/lib/python3.8/dist-packages/torch/utils/cpp_extension.py:387: UserWarning: The detected CUDA version (11.2) has a minor version mismatch with the version that was used to compile PyTorch (11.7). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
building '_ext' extension
creating /content/yolact/external/DCNv2/build
creating /content/yolact/external/DCNv2/build/temp.linux-x86_64-3.8
creating /content/yolact/external/DCNv2

## Pretrained Weights
In order to run inference, we'll need some pre-trained weights. The creator of the GitHub repo shared them on Google Drive. We're going to use a [helpful tool](https://github.com/chentinghao/download_google_drive) made by [chentinghao](https://github.com/chentinghao) to easily access the Drive file from Colab.

If this stops working, it's probably because the weights moved to a different location. Check the YOLACT github repo to see where they went.

In [40]:
# Make sure we're in the top folder
%cd /content

# Clone the repo
!git clone https://github.com/chentinghao/download_google_drive.git

# Create a new directory for the pre-trained weights
!mkdir -p /content/yolact/weights

# Download the file
!python ./download_google_drive/download_gdrive.py 1ZPu1YR2UzGHQD0o1rEqy-j5bmEm3lbyP ./yolact/weights/yolact_plus_resnet50_54_800000.pth

/content
fatal: destination path 'download_google_drive' already exists and is not an empty directory.
32.0kB [00:00, 107MB/s]


# Get Test Images
We'll download a few test images from the [COCO dataset](http://cocodataset.org/#explore). Feel free to try out your own images as well, but know that there are only 80 categories and they're not intended to cover everything, so if you upload a picture of a snake, a mouse, and a cat, only the cat will be detected.

In [19]:
# Make a new directory for the test images
!mkdir /content/test_images

# Move to the new directory
%cd /content/test_images

# Make a list of URLs from the COCO dataset
get_imgs = ['http://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg',
            'http://farm3.staticflickr.com/2835/9520716824_51220b13d0_z.jpg',
            'http://farm3.staticflickr.com/2817/10105052924_0c8d79e62c_z.jpg',
            'https://farm3.staticflickr.com/2637/3679968895_37a79e55b3_z.jpg',
            'https://farm8.staticflickr.com/7137/7705349298_6bea29cf1c_z.jpg',
            'http://farm5.staticflickr.com/4105/4981945644_3f162314ed_z.jpg',
            'http://farm1.staticflickr.com/40/101757885_a8fc40e273_z.jpg',
            'http://farm3.staticflickr.com/2654/3897535041_fa8dd5570b_z.jpg']

# Download all the images
for img_url in get_imgs:
  !wget {img_url} -nc

# List the contents of the folder. Should have images.
!pwd
!ls

/content/test_images
--2023-01-04 20:00:32--  http://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg
Resolving farm8.staticflickr.com (farm8.staticflickr.com)... 13.33.38.82, 2600:9000:229f:aa00:0:5a51:64c9:c681, 2600:9000:229f:ae00:0:5a51:64c9:c681, ...
Connecting to farm8.staticflickr.com (farm8.staticflickr.com)|13.33.38.82|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg [following]
--2023-01-04 20:00:32--  https://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg
Connecting to farm8.staticflickr.com (farm8.staticflickr.com)|13.33.38.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [image/jpeg]
Saving to: ‘6946559805_7edf757660_z.jpg’

6946559805_7edf7576     [ <=>                ]  52.93K  --.-KB/s    in 0.002s  

2023-01-04 20:00:32 (26.3 MB/s) - ‘6946559805_7edf757660_z.jpg’ saved [54199]

--2023-01-04 20:00:32--  http://f

# Run Inference on Images
Let's run inference. We do that with eval.py

In [62]:
# Move up to the top level directory
%cd /content

# Delete any previous output_images folder (in case we're running this more than once)
!rm /content/output_images -r

# Create a new directory for output images
!mkdir -p /content/output_images

# Run inference using our pre-trained weights on all images in the directory
!python ./yolact/eval.py --trained_model=./yolact/weights/yolact_plus_resnet50_54_800000.pth --config=yolact_plus_resnet50_config --score_threshold=0.15 --top_k=15 --images=test_images:output_images

/content
Loading model...Traceback (most recent call last):
  File "./yolact/eval.py", line 1097, in <module>
    net = Yolact()
  File "/content/yolact/yolact.py", line 402, in __init__
    self.backbone = construct_backbone(cfg.backbone)
  File "/content/yolact/backbone.py", line 451, in construct_backbone
    backbone = cfg.type(*cfg.args)
  File "/content/yolact/backbone.py", line 83, in __init__
    self._make_layer(block, 128, layers[1], stride=2, dcn_layers=dcn_layers[1], dcn_interval=dcn_interval)
  File "/content/yolact/backbone.py", line 114, in _make_layer
    layers.append(block(self.inplanes, planes, stride, downsample, self.norm_layer, self.dilation, use_dcn=use_dcn))
  File "/content/yolact/backbone.py", line 22, in __init__
    self.conv2 = DCN(planes, planes, kernel_size=3, stride=stride,
  File "/content/yolact/backbone.py", line 11, in DCN
    raise Exception('DCN could not be imported. If you want to use YOLACT++ models, compile DCN. Check the README for instruction

## Display Output Images
The last step saved output images, so we'll display those now.

In [63]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

output_images = Path('output_images')

def show_image(img_path):
  img = cv2.imread(img_path)
  img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.figure(figsize=(16,16))
  plt.imshow(img_cvt)
  plt.show()

# Iterate through all of the output images and display them
for img_path in output_images.iterdir():
  print(img_path)
  show_image(str(img_path))

If you'd like to process your own video, you can modify the steps below.

In [ ]:
# Make a new directory for the test images
!mkdir -p /content/test_video

# Move to the top level directory
%cd /content

# Download the file
# https://drive.google.com/file/d/1I4fivRLniVNt_LcWqhQNehTSdEkZ4ytW/view?usp=sharing
# Note that file_id is extracted from the share URL above, it'll be different for every file
file_id = "1I4fivRLniVNt_LcWqhQNehTSdEkZ4ytW"
file_path = "/content/test_video/giraffes_full_720.mp4" # <- this is the destination path

!python /content/download_google_drive/download_gdrive.py {file_id} {file_path}

!ls /content/test_video

Now we run inference.

In [64]:
%cd /content
!mkdir -p /content/output_videos
output_path = "/content/output_videos/giraffes_full_720_output.mp4"
!python ./yolact/eval.py --trained_model=./yolact/weights/yolact_plus_resnet50_54_800000.pth --score_threshold=0.15 --top_k=15 --video_multiframe=4 --video={file_path}:{output_path}

/content
Config not specified. Parsed yolact_plus_resnet50_config from the file name.

Loading model...Traceback (most recent call last):
  File "./yolact/eval.py", line 1097, in <module>
    net = Yolact()
  File "/content/yolact/yolact.py", line 402, in __init__
    self.backbone = construct_backbone(cfg.backbone)
  File "/content/yolact/backbone.py", line 451, in construct_backbone
    backbone = cfg.type(*cfg.args)
  File "/content/yolact/backbone.py", line 83, in __init__
    self._make_layer(block, 128, layers[1], stride=2, dcn_layers=dcn_layers[1], dcn_interval=dcn_interval)
  File "/content/yolact/backbone.py", line 114, in _make_layer
    layers.append(block(self.inplanes, planes, stride, downsample, self.norm_layer, self.dilation, use_dcn=use_dcn))
  File "/content/yolact/backbone.py", line 22, in __init__
    self.conv2 = DCN(planes, planes, kernel_size=3, stride=stride,
  File "/content/yolact/backbone.py", line 11, in DCN
    raise Exception('DCN could not be imported. If 

## Download the output file
In this part, we'll mount your Google Drive and you can copy the file somewhere over there. I tried using the google.colab.files library to download the file, but it doesn't seem to like big video files.

- You will need to authorize your Drive to connect, then it basically just links your drive as if it's a folder on this cloud computer.
- **Be careful** because if you start deleting/modifying files in this folder, you will modify your personal Google Drive.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir -p /content/drive/My\ Drive/YOLACT_output
%cd /content/drive/My\ Drive/YOLACT_output
!ls

In [ ]:
!cp {output_path} /content/drive/My\ Drive/YOLACT_output/giraffes_full_720_output.mp4